In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "c_a_tunes_dev")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_invoices_transformed = spark.table(f"{catalogo}.{esquema_source}.invoices_transformed")


In [0]:
df_invoices_transformed.head(1)

In [0]:
df_transformed = df_invoices_transformed.groupBy(col("Track_Name"), col("Invoice_Date"), col("Invoice_Id"), col("Amount_type")) \
    .agg(
        count(col("Invoice_Line_Id")).alias("conteo"),
        count(col("Track_Id")).alias("tracks"),
        max(col("Unit_Price")).alias("max_Price"),
        min(col("Unit_Price")).alias("min_Price"),
        max(col("Quantity")).alias("max_quantity"),
        sum((col("Unit_Price")*col("Quantity"))).alias("total")
    ).orderBy(col("Invoice_Date").desc(), col("Track_Name").desc(), col("Invoice_Id").desc())

In [0]:
##df_transformed.head(1)

In [0]:
%sql
DROP TABLE IF EXISTS c_a_tunes_dev.golden.golden_invoice_partitioned;

In [0]:
df_transformed.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.golden_invoice_partitioned")